# 理解问题
## 问题的定义
本工程是一个二分类问题，针对提供96\*96px的图片给出中心32\*32px图片有肿瘤的概率（0-1）

## 预期的结果输出
对于二分类的问题，常见的结果输出有
* AUC
* logloss
* accuracy
* precision

AUC <br>
AUC就是从1的样本中随机取一个，从0的样本中随机取一个，把1的样本预测为1的概率为p1，把0的样本预测为1的概率为p0, p1>p0的概率就是AUC。另外值得注意的是，AUC对样本类别是否均衡并不敏感，这也是不均衡样本通常用AUC评价分类器性能的一个原因。

## 背景知识了解



# 获取数据
## 读取数据

### train_label中的01二分类的分布

In [56]:
import numpy as np
import pandas as pd
import os
import cv2
import matplotlib.pyplot as plt
import matplotlib.patches as patches
import random
from sklearn.utils import shuffle
from tqdm import tqdm_notebook


data = pd.read_csv('/kaggle/input/train_labels.csv')
train_path = '/kaggle/input/train/'
test_path = '/kaggle/input/test/'
# 检验输入的标签分布
data['label'].value_counts()

0    130908
1     89117
Name: label, dtype: int64

### train_label中存取的数据

In [ ]:
data.head()

### train目录下的文件

In [ ]:
os.listdir(train_path)

### 读取图片

In [ ]:
def readImage(path):
    bgr_img = cv2.imread(path)
    b,g,r = cv2.split(bgr_img)
    rgb_img = cv2.merge([r,g,b])
    return rgb_img

shuffled_data = shuffle(data)

fig, ax = plt.subplots(2,5, figsize=(20,8))
fig.suptitle('picture',fontsize=20)
# 无肿瘤
for i, idx in enumerate(shuffled_data[shuffled_data['label'] == 0]['id'][:5]):
    path = os.path.join(train_path, idx)
    ax[0,i].imshow(readImage(path + '.tif'))
    # 框出中心32*32
    box = patches.Rectangle((32,32),32,32,linewidth=4,edgecolor='b',facecolor='none', linestyle=':', capstyle='round')
    ax[0,i].add_patch(box)
ax[0,0].set_ylabel('NONE tumor sample', size='large')
# 有肿瘤
for i, idx in enumerate(shuffled_data[shuffled_data['label'] == 1]['id'][:5]):
    path = os.path.join(train_path, idx)
    ax[1,i].imshow(readImage(path + '.tif'))
    # 获取正方形的区域
    box = patches.Rectangle((32,32),32,32,linewidth=4,edgecolor='r',facecolor='none', linestyle=':', capstyle='round')
    ax[1,i].add_patch(box)
ax[1,0].set_ylabel('tumor sample', size='large')

# 特征工程
## 图片截取


## 图片增强

In [ ]:
import random
ORIGINAL_SIZE = 96      # 原始图片的大小

# 增强的变量
CROP_SIZE = 96         # 图片截取之后的大小
RANDOM_ROTATION = 3    # 旋转范围(0-180),这里选择旋转3度 
RANDOM_SHIFT = 2        # center crop shift in x and y axes, 0=no change. This cannot be more than (ORIGINAL_SIZE - CROP_SIZE)//2 
RANDOM_BRIGHTNESS = 7  # 亮度增强
RANDOM_CONTRAST = 5    # 亮度对比
RANDOM_90_DEG_TURN = 1  # 0 or 1= 向左或者向右旋转

def readCroppedImage(path, augmentations = True):
    
    # opencv 按照bgr的格式读取数据
    bgr_img = cv2.imread(path)
    # 将三路信号分割
    b,g,r = cv2.split(bgr_img)
    rgb_img = cv2.merge([r,g,b])
    
    if(not augmentations):
        return rgb_img / 255
    
    #随机旋转
#     rotation = random.randint(-RANDOM_ROTATION,RANDOM_ROTATION)
#     if(RANDOM_90_DEG_TURN == 1):
#         rotation += random.randint(-1,1) * 90
#     M = cv2.getRotationMatrix2D((48,48),rotation,1)   # 中心点是旋转的中心
#     rgb_img = cv2.warpAffine(rgb_img,M,(96,96))
    
#     #x,y 轴的平移
#     x = random.randint(-RANDOM_SHIFT, RANDOM_SHIFT)
#     y = random.randint(-RANDOM_SHIFT, RANDOM_SHIFT)
    x = 0
    y = 0
    
    # 将图片进行截取
    start_crop = (ORIGINAL_SIZE - CROP_SIZE) // 2
    end_crop = start_crop + CROP_SIZE
    rgb_img = rgb_img[(start_crop + x):(end_crop + x), (start_crop + y):(end_crop + y)] / 255
    
#     # 图片翻转
#     flip_hor = bool(random.getrandbits(1))
#     flip_ver = bool(random.getrandbits(1))
#     if(flip_hor):
#         rgb_img = rgb_img[:, ::-1]
#     if(flip_ver):
#         rgb_img = rgb_img[::-1, :]
        
#     # 改变亮度
#     br = random.randint(-RANDOM_BRIGHTNESS, RANDOM_BRIGHTNESS) / 100.
#     rgb_img = rgb_img + br
    
#     # 改变对比度
#     cr = 1.0 + random.randint(-RANDOM_CONTRAST, RANDOM_CONTRAST) / 100.
#     rgb_img = rgb_img * cr
    
    # 将rgb值映射到0-1之间
    rgb_img = np.clip(rgb_img, 0, 1.0)
    
    return rgb_img

In [ ]:
fig, ax = plt.subplots(2,5, figsize=(20,8))
fig.suptitle('淋巴组织图片截取',fontsize=20)
# 阴性组织
for i, idx in enumerate(shuffled_data[shuffled_data['label'] == 0]['id'][:5]):
    path = os.path.join(train_path, idx)
    ax[0,i].imshow(readCroppedImage(path + '.tif'))
    box = patches.Rectangle((32,32),32,32,linewidth=4,edgecolor='b',facecolor='none', linestyle=':', capstyle='round')
    ax[0,i].add_patch(box)
ax[0,0].set_ylabel('阴性样本', size='large')
# Positives
for i, idx in enumerate(shuffled_data[shuffled_data['label'] == 1]['id'][:5]):
    path = os.path.join(train_path, idx)
    ax[1,i].imshow(readCroppedImage(path + '.tif'))
    box = patches.Rectangle((32,32),32,32,linewidth=4,edgecolor='r',facecolor='none', linestyle=':', capstyle='round')
    ax[1,i].add_patch(box)
ax[1,0].set_ylabel('阳性样本', size='large')

检查对同一张图片的变换

In [ ]:
fig, ax = plt.subplots(1,5, figsize=(20,4))
fig.suptitle('对图片做增强的结果',fontsize=20)
# 阴性
for i, idx in enumerate(shuffled_data[shuffled_data['label'] == 0]['id'][:1]):
    for j in range(5):
        path = os.path.join(train_path, idx)
        ax[j].imshow(readCroppedImage(path + '.tif'))

数据清洗

In [ ]:
# 对数据进行清洗，检查是否有过于明亮或者过于暗淡的数据
dark_th = 10 / 255      # 若一张图片的所有像素点的r,g,b之中都没有高于这个值的，则认为这张图片过暗
bright_th = 245 / 255   # 若一张图片的所有像素点的r,g,b之中都没有低于这个值的，则认为这张图片过亮
too_dark_idx = []
too_bright_idx = []

x_tot = np.zeros(3)
x2_tot = np.zeros(3)
counted_ones = 0
shuffled_data = shuffled_data.sample(10000, random_state = 101)
data = shuffled_data
for i, idx in tqdm_notebook(enumerate(shuffled_data['id']), 'computing statistics.(220025 it total)'):
    path = os.path.join(train_path, idx)
    imagearray = readCroppedImage(path + '.tif', augmentations = False).reshape(-1,3)
    # 这张图太暗
    if(imagearray.max() < dark_th):
        too_dark_idx.append(idx)
        continue # 不会将这张图片加入到数据集中
    # 这张图太亮
    if(imagearray.min() > bright_th):
        too_bright_idx.append(idx)
        continue # 不会将这张图加入到数据集中
    x_tot += imagearray.mean(axis=0)
    x2_tot += (imagearray**2).mean(axis=0)
    counted_ones += 1
    
channel_avr = x_tot/counted_ones # 求解均值
channel_std = np.sqrt(x2_tot/counted_ones - channel_avr**2) #求解方差
print(channel_avr,channel_std)

In [ ]:
print('{0}张图片太暗'.format(len(too_dark_idx)))
print('{0}张图片太亮'.format(len(too_bright_idx)))
print('暗的图片:')
print(too_dark_idx)
print('亮的图片:')
print(too_bright_idx)

In [ ]:
from sklearn.model_selection import train_test_split
train_df = data

print('在移除不合格图片之前我们有{0}训练样本'.format(len(train_df)))
for i in too_dark_idx:
    train_df =  train_df[train_df['id'] != i]
    
for j in too_bright_idx:
    train_df =  train_df[train_df['id'] != j]

print('在移除不合格图片后我们有{0}训练样本.'.format(len(train_df)))

train_names = train_df.id.values
train_labels = np.asarray(train_df['label'].values)

# 将数据分割为训练集和测试集
df_train, df_val= train_test_split(train_df, test_size=0.1, stratify=train_labels, random_state=123)


### 训练集大小

In [ ]:
df_train.shape

In [ ]:
df_train['label'].value_counts()

In [ ]:
df_val['label'].value_counts()
print(os.listdir('../'))

In [ ]:
import shutil

In [ ]:
shutil.rmtree('base_dir') 

In [ ]:
base_dir = 'base_dir'
os.mkdir(base_dir)

In [ ]:
#在base_dir文件夹中创建文件夹]

# 在'base_dir'创建两个文件夹:

# train_dir
    # a_no_tumor_tissue
    # b_has_tumor_tissue

# val_dir
    # a_no_tumor_tissue
    # b_has_tumor_tissue
    
# 文件路径连接到一起
# train_dir
train_dir = os.path.join(base_dir, 'train_dir')
os.mkdir(train_dir)

# val_dir
val_dir = os.path.join(base_dir, 'val_dir')
os.mkdir(val_dir)



# [在训练集和测试集中创建文件夹]
# 我们为阴性图片与阳性图片创建文件夹

# 在训练集中创建文件夹
no_tumor_tissue = os.path.join(train_dir, 'a_no_tumor_tissue')
os.mkdir(no_tumor_tissue)
has_tumor_tissue = os.path.join(train_dir, 'b_has_tumor_tissue')
os.mkdir(has_tumor_tissue)


# 在验证集中创建文件夹
no_tumor_tissue = os.path.join(val_dir, 'a_no_tumor_tissue')
os.mkdir(no_tumor_tissue)
has_tumor_tissue = os.path.join(val_dir, 'b_has_tumor_tissue')
os.mkdir(has_tumor_tissue)

In [ ]:
os.listdir("../working/base_dir")

In [ ]:
os.listdir('base_dir/train_dir')

In [ ]:
train_df.head()

In [ ]:
# Set the id as the index in df_data
df_data = train_df
df_data.set_index('id', inplace=True)

In [ ]:
os.listdir("/kaggle")

In [ ]:
df_train.head()

In [ ]:
train_list = list(df_train['id'])
val_list = list(df_val['id'])



# 将测试数据转移到指定文件夹下

for image in train_list:
    
    # csv文件夹中文件没有.tif格式后缀，我们在这里加上
    fname = image + '.tif'
    # 获取指定图片的label
    target = df_data.loc[image,'label']
    infnamepath = '../input/train/'+fname
    imagearray = readCroppedImage(infnamepath)
    # 图片名称与它们lable相对应
    if target == 0:
        label = 'a_no_tumor_tissue'
    if target == 1:
        label = 'b_has_tumor_tissue'
    
    # 原路径
    src = os.path.join('../input/train', fname)
    # 图片的目标路径
    dst = os.path.join(train_dir, label, fname)
    # 将图片复制到指定路径下
    shutil.copyfile(src, dst)
    outfnamepath = train_dir+'/'+label+'/'+fname
    r,g,b = cv2.split((imagearray * 255).astype(int)) 
    bgr_img = cv2.merge([b,g,r])
    cv2.imwrite(outfnamepath, bgr_img)


# 将验证集转移到到指定文件夹下

for image in val_list:
    
    # csv文件夹中文件没有.tif格式后缀，我们在这里加上
    fname = image + '.tif'
    infnamepath = '/kaggle/input/train/'+fname
    imagearray = readCroppedImage(infnamepath)
    # 获取指定图片的lable
    target = df_data.loc[image,'label']
    
    # 图片的lable必须与文件名对应
    if target == 0:
        label = 'a_no_tumor_tissue'
    if target == 1:
        label = 'b_has_tumor_tissue'
    

    # 图片源地址
    src = os.path.join('/kaggle/input/train', fname)
    # 图片目的地址
    dst = os.path.join(val_dir, label, fname)
    # 将图片从源地址拷贝到目的地址
    shutil.copyfile(src, dst)
    outfnamepath = val_dir+'/'+label+'/'+fname
    r,g,b = cv2.split((imagearray * 255).astype(int)) 
    bgr_img = cv2.merge([b,g,r])
    cv2.imwrite(outfnamepath, bgr_img)
    

In [ ]:
os.listdir('/kaggle/working/base_dir/train_dir/a_no_tumor_tissue')

In [ ]:
# 输出每个文件夹有多少图片

print(len(os.listdir('base_dir/train_dir/a_no_tumor_tissue')))
print(len(os.listdir('base_dir/train_dir/b_has_tumor_tissue')))

In [ ]:
# 输出每个文件夹有多少验证图片

print(len(os.listdir('base_dir/val_dir/a_no_tumor_tissue')))
print(len(os.listdir('base_dir/val_dir/b_has_tumor_tissue')))

In [ ]:
train_path = 'base_dir/train_dir'
valid_path = 'base_dir/val_dir'
test_path = '../input/test'

num_train_samples = len(df_train)
num_val_samples = len(df_val)
train_batch_size = 10
val_batch_size = 10


train_steps = np.ceil(num_train_samples / train_batch_size)
val_steps = np.ceil(num_val_samples / val_batch_size)


In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator
IMAGE_SIZE = 96

In [ ]:
datagen = ImageDataGenerator(rescale=1.0/255)

train_gen = datagen.flow_from_directory(train_path,
                                        target_size=(IMAGE_SIZE,IMAGE_SIZE),
                                        batch_size=train_batch_size,
                                        class_mode='categorical')

val_gen = datagen.flow_from_directory(valid_path,
                                        target_size=(IMAGE_SIZE,IMAGE_SIZE),
                                        batch_size=val_batch_size,
                                        class_mode='categorical')


test_gen = datagen.flow_from_directory(valid_path,
                                        target_size=(IMAGE_SIZE,IMAGE_SIZE),
                                        batch_size=1,
                                        class_mode='categorical',
                                        shuffle=False)

In [ ]:
from numpy.random import seed
seed(101)
from tensorflow import set_random_seed
set_random_seed(101)

import pandas as pd
import numpy as np


import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Dense, Dropout, Flatten, Activation
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras.optimizers import Adam

import os
import cv2

from sklearn.utils import shuffle
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split
import itertools
import shutil
import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
from keras import regularizers
kernel_size = (3,3)
pool_size= (2,2)
pool_size1 = (1,1)
first_filters = 32
second_filters = 64
third_filters = 128

dropout_conv = 0.3
dropout_dense = 0.3


model = Sequential()
model.add(Conv2D(first_filters, kernel_size, activation = 'relu', input_shape = (IMAGE_SIZE, IMAGE_SIZE, 3), kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l1(0.01)))
model.add(Conv2D(first_filters, kernel_size, activation = 'relu',kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l1(0.01)))
model.add(Conv2D(first_filters, kernel_size, activation = 'relu',kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l1(0.01)))
model.add(MaxPooling2D(pool_size = pool_size)) 
model.add(Dropout(dropout_conv))

model.add(Conv2D(second_filters, kernel_size, activation ='relu',kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l1(0.01)))
model.add(Conv2D(second_filters, kernel_size, activation ='relu',kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l1(0.01)))
model.add(Conv2D(second_filters, kernel_size, activation ='relu',kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l1(0.01)))
model.add(MaxPooling2D(pool_size = pool_size1))
model.add(Dropout(dropout_conv))

model.add(Conv2D(third_filters, kernel_size, activation ='relu',kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l1(0.01)))
model.add(Conv2D(third_filters, kernel_size, activation ='relu',kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l1(0.01)))
model.add(Conv2D(third_filters, kernel_size, activation ='relu',kernel_regularizer=regularizers.l2(0.01),
                activity_regularizer=regularizers.l1(0.01)))
model.add(MaxPooling2D(pool_size = pool_size1))
model.add(Dropout(dropout_conv))

model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(dropout_dense))
model.add(Dense(2, activation = "softmax"))

model.summary()

In [ ]:
model.compile(Adam(lr=0.0001), loss='binary_crossentropy', 
              metrics=['accuracy'])

In [ ]:
print(val_gen.class_indices)

In [ ]:
os.remove('model.h5')

In [ ]:
filepath = "model.h5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, 
                             save_best_only=True, mode='max')

reduce_lr = ReduceLROnPlateau(monitor='val_acc', factor=0.5, patience=2, 
                                   verbose=1, mode='max', min_lr=0.00001)
                              
                              
callbacks_list = [checkpoint, reduce_lr]

history = model.fit_generator(train_gen, steps_per_epoch=train_steps, 
                    validation_data=val_gen,
                    validation_steps=val_steps,
                    epochs=20, verbose=1,
                   callbacks=callbacks_list)

In [ ]:
# get the metric names so we can use evaulate_generator
model.metrics_names

In [ ]:
# Here the best epoch will be used.

model.load_weights('model.h5')

val_loss, val_acc = \
model.evaluate_generator(test_gen, 
                        steps=len(df_val))

print('val_loss:', val_loss)
print('val_acc:', val_acc)

In [ ]:
# display the loss and accuracy curves

import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(1, len(acc) + 1)

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.figure()

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()

In [ ]:
predictions = model.predict_generator(test_gen, steps=len(df_val), verbose=1)

In [ ]:
predictions.shape

In [ ]:
# This is how to check what index keras has internally assigned to each class. 
test_gen.class_indices

In [ ]:
# Put the predictions into a dataframe.
# The columns need to be oredered to match the output of the previous cell

df_preds = pd.DataFrame(predictions, columns=['no_tumor_tissue', 'has_tumor_tissue'])

df_preds.head()

In [ ]:
# Get the true labels
y_true = test_gen.classes

# Get the predicted labels as probabilities
y_pred = df_preds['has_tumor_tissue']

In [ ]:
from sklearn.metrics import roc_auc_score

roc_auc_score(y_true, y_pred)

In [ ]:
# Source: Scikit Learn website
# http://scikit-learn.org/stable/auto_examples/
# model_selection/plot_confusion_matrix.html#sphx-glr-auto-examples-model-
# selection-plot-confusion-matrix-py


def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.ylabel('True label')
    plt.xlabel('Predicted label')
    plt.tight_layout()

In [ ]:
# Get the labels of the test images.

test_labels = test_gen.classes

In [ ]:
test_labels.shape

In [ ]:
# argmax returns the index of the max value in a row
cm = confusion_matrix(test_labels, predictions.argmax(axis=1))

In [ ]:
# Print the label associated with each class
test_gen.class_indices

In [ ]:
# Define the labels of the class indices. These need to match the 
# order shown above.
cm_plot_labels = ['no_tumor_tissue', 'has_tumor_tissue']

plot_confusion_matrix(cm, cm_plot_labels, title='Confusion Matrix')

In [ ]:
from sklearn.metrics import classification_report

# Generate a classification report

# For this to work we need y_pred as binary labels not as probabilities
y_pred_binary = predictions.argmax(axis=1)

report = classification_report(y_true, y_pred_binary, target_names=cm_plot_labels)

print(report)

In [ ]:
# Delete base_dir and it's sub folders to free up disk space.

shutil.rmtree('base_dir')

In [ ]:
#[CREATE A TEST FOLDER DIRECTORY STRUCTURE]

# We will be feeding test images from a folder into predict_generator().
# Keras requires that the path should point to a folder containing images and not
# to the images themselves. That is why we are creating a folder (test_images) 
# inside another folder (test_dir).

# test_dir
    # test_images

# create test_dir
test_dir = 'test_dir'
os.mkdir(test_dir)
    
# create test_images inside test_dir
test_images = os.path.join(test_dir, 'test_images')
os.mkdir(test_images)

In [ ]:
os.listdir('test_dir')

In [ ]:
# Transfer the test images into image_dir

test_list = os.listdir('../input/test')

for image in test_list:
    
    fname = image
    
    # source path to image
    src = os.path.join('../input/test', fname)
    # destination path to image
    dst = os.path.join(test_images, fname)
    # copy the image from the source to the destination
    shutil.copyfile(src, dst)
    imagearray = readCroppedImage(src)
    r,g,b = cv2.split((imagearray * 255).astype(int)) 
    bgr_img = cv2.merge([b,g,r])
    cv2.imwrite(dst, bgr_img)
    

In [ ]:
# check that the images are now in the test_images
# Should now be 57458 images in the test_images folder

len(os.listdir('test_dir/test_images'))

In [ ]:
test_path ='test_dir'


# Here we change the path to point to the test_images folder.

test_gen = datagen.flow_from_directory(test_path,
                                        target_size=(IMAGE_SIZE,IMAGE_SIZE),
                                        batch_size=1,
                                        class_mode='categorical',
                                        shuffle=False)

In [ ]:
num_test_images = 57458

# make sure we are using the best epoch
model.load_weights('model.h5')

predictions = model.predict_generator(test_gen, steps=num_test_images, verbose=1)

In [ ]:
# Are the number of predictions correct?
# Should be 57458.

len(predictions)

In [ ]:
# Put the predictions into a dataframe

df_preds = pd.DataFrame(predictions, columns=['no_tumor_tissue', 'has_tumor_tissue'])

df_preds.head()

In [ ]:
# This outputs the file names in the sequence in which 
# the generator processed the test images.
test_filenames = test_gen.filenames

# add the filenames to the dataframe
df_preds['file_names'] = test_filenames

df_preds.head()

In [ ]:
# Create an id column

# A file name now has this format: 
# test_images/00006537328c33e284c973d7b39d340809f7271b.tif

# This function will extract the id:
# 00006537328c33e284c973d7b39d340809f7271b


def extract_id(x):
    
    # split into a list
    a = x.split('/')
    # split into a list
    b = a[1].split('.')
    extracted_id = b[0]
    
    return extracted_id

df_preds['id'] = df_preds['file_names'].apply(extract_id)

df_preds.head()

In [ ]:
# Get the predicted labels.
# We were asked to predict a probability that the image has tumor tissue
y_pred = df_preds['has_tumor_tissue']

# get the id column
image_id = df_preds['id']

In [ ]:
submission = pd.DataFrame({'id':image_id, 
                           'label':y_pred, 
                          }).set_index('id')

submission.to_csv('patch_preds.csv', columns=['label'])

In [ ]:
# Delete the test_dir directory we created to prevent a Kaggle error.
# Kaggle allows a max of 500 files to be saved.

shutil.rmtree('test_dir')